In [6]:
from django.db.models import Model
from django.shortcuts import render,redirect
from ecommerce.models import *
import pandas as pd
import json
from django.http import JsonResponse

AppRegistryNotReady: Apps aren't loaded yet.

In [ ]:
def create_inbound_document(mock_inbound):
    
    for i in range(len(mock_inbound)):
        sellers = mock_inbound[i]['seller']
        products = mock_inbound[i]['product']
        codes = mock_inbound[i]['code']
        qtys = mock_inbound[i]['quantity']
        product = Product.objects.get(pk=products)
        seller = Seller.objects.get(pk=sellers)
        Inbound.objects.create(
            seller = seller,
            item = product,
            code = codes,
            qty = qtys
        )
        
        move = stock_move(product,qtys)
        print("เพิ่มสินค้า",product.name,"ใน Inbound ", qtys ,"ชื้น")
        
create_inbound_document(mock_inbound=[{'seller':1,'product':1,'code': 'A0001','quantity':100}])

In [ ]:
def create_inbound_document(mock_inbound):
    
    for i in range(len(mock_inbound)):
        sellers = mock_inbound[i]['seller']
        products = mock_inbound[i]['product']
        codes = mock_inbound[i]['code']
        qtys = mock_inbound[i]['quantity']
        product = Product.objects.get(pk=products)
        seller = Seller.objects.get(pk=sellers)
        Inbound.objects.create(
            seller = seller,
            item = product,
            code = codes,
            qty = qtys
        )
        
        move = stock_move(product,qtys)
        print("เพิ่มสินค้า",product.name,"ใน Inbound ", qtys ,"ชื้น")

    
def stock_move(product,qtys):
    src_location = Location.objects.get(pk=1)
    des_location = Location.objects.get(pk=2)
    return Move.objects.create(Product=product,qty=qtys,src_location=src_location,des_location=des_location)

def check_inbound_quality(mock_product):
    pk_src=1
    pk_des=2
    for i in range(len(mock_product)):
        products = mock_product[i]['product']
        qtys = mock_product[i]['qty']
        product = Product.objects.get(pk=products)
        actives = Inbound.objects.get(item=product)
        passed = check_quality(product,actives.active)
        pk_src +=1
        pk_des +=1
        if passed == True:
            src_location = Location.objects.get(pk=pk_src)
            des_location = Location.objects.get(pk=pk_des)
            move_l1 = Move.objects.create(
                Product=product,
                qty=qtys,
                src_location=src_location,
                des_location=des_location
            )
        print(src_location.location, " มีจำนวนสินค้าคงเหลือ ", qtys, product.unit)

        
def check_quality(product,actives):
    inbound = Inbound.objects.get(item=product)
    if actives == True:
        return True
    else:
        return False
    


def checkout_shipping_cart(cart):
    
    customer = [{
        'Profile':1,
        'type_order':"Lazada"
    }
        
    ]
    profiles = customer[0]['Profile']
    type_order= customer[0]['type_order']
    profile = Profile.objects.get(pk=profiles)
    lists=SaleOrder.objects.create(
        cust_profile = profile,
        type_order = type_order
    )
    sale_order = SaleOrder.objects.all().last()
    for i in range(len(cart)):
        item = cart[i]['item']
        qty = cart[i]['qty']
        product = Product.objects.get(pk=item)
        OrderItem.objects.create(
            sale_order = sale_order,
            item=product,
            qty=qty
        )
        print("รหัส SaleOder",lists ,"สินค้า", product.name ,"จำนวน",qty,"ชิ้น")

def create_picking_list(sale_orders):
    PickRequest.objects.create()
    pick_request = PickRequest.objects.all().last()
    
    for i in sale_orders:
        sale_order = SaleOrder.objects.get(pk=i)
        SaleOrder.objects.filter(pk=sale_order.pk).update(picking_request=pick_request)
        
    order_items = OrderItem.objects.filter(sale_order__pk__in=sale_orders).values('item').annotate(sum_qty=Sum('qty'))
    for item in order_items:
        product = Product.objects.get(pk=item['item'])
        PickItem.objects.create(
            pick_request=pick_request,
            item=product,      #ตัวหลัง คือตัวแปร ตัวหน้า คือ ตัวในตาราง
            qty=item['sum_qty']
            )
        
    listpick=list_pick(pick_request,sale_orders)    
 
    
    return print(listpick)

def list_pick(pick_request,sale_orders):
    dicts={}
    recodes=[]
    pick_request={'pick_request':pick_request.pk}
    recodes.append(pick_request)
    for i in  sale_orders:
        saleorder = SaleOrder.objects.filter(pk=i).values(
                    'orderitem__id',
                    'orderitem__item__name',
                    'orderitem__qty'
                    )
        so = {'sale_order_id': i  }
        recodes.append(so)
        for i in saleorder:
            recode = {'item':{'order_item_id ': i['orderitem__id'],'product': i['orderitem__item__name'] , 'qty': i['orderitem__qty']}}
            recodes.append(recode)
        dicts=recodes
    return (dicts)

            
def create_packing_list(sale_orders):
  
    for sale in sale_orders:
        sale_order = SaleOrder.objects.get(pk=sale)
        profile = Profile.objects.get(pk=sale_order.cust_profile.pk)
        PackRequest.objects.create(
            sale_order = SaleOrder.objects.get(pk=sale),
            customer_profile= profile
        )
        pack_request= PackRequest.objects.all().last()
        list_order_item = OrderItem.objects.filter(sale_order=sale_order)
        for lists in range(len(list_order_item)):
            product = Product.objects.get(pk=list_order_item[lists].item.pk)
            PackItem.objects.create(
            pack_request=pack_request,
            item=product,
            qty=list_order_item[lists].qty
            )
        listpack = list_pack(pack_request,sale)
        print(listpack)
        
            
def list_pack(pack_request,sale):
    dicts={}
    recodes=[]
    saleorder = PackItem.objects.filter(pack_request =pack_request).values(
        'id',
        'item__name',
        'qty'
        )
    pack_request={'pack_request':pack_request.pk}
    so = {'sale_order_id': sale  }
    recodes.append(pack_request)
    recodes.append(so)
    for i in saleorder:
        recode = {'item':{'order_item_id ': i['id'],'product': i['item__name'] , 'qty': i['qty']}}
        recodes.append(recode)
    dicts=recodes
    return (dicts)


            
def pick_list():
    pick_list=PickItem.objects.all()
    for i in pick_list:
        print("ID :",i.pk," PICKREQUEST :",i.pick_request," PRODUCT :",i.item,"QUANTITY :",i.qty)

def pack_list():
    pack_list=PackItem.objects.all()
    for i in pack_list:
        print("ID :",i.pk," PACKREQUEST :",i.pack_request," PRODUCT :",i.item,"QUANTITY :",i.qty)


def list_stock(product=None):
    if product==None:
        list_pick_item= PickItem.objects.values('item__id','item__name').order_by('item').annotate(sum_qty=Sum('qty'))
        for item in list_pick_item:
            inbound = Inbound.objects.get(item=item['item__id'])
            updateqty = inbound.qty-item['sum_qty']
            print(" id:",inbound.id," seller:",inbound.seller," Product:",inbound.item," Code:",inbound.code," Active:",inbound.active," Qty:",updateqty)
    else:
        products = Product.objects.get(pk=product)
        list_pick_item= PickItem.objects.filter(item=products).values('item__id','item__name').order_by('item').annotate(sum_qty=Sum('qty'))
        for item in list_pick_item:
            inbound = Inbound.objects.get(item=item['item__id'])
            updateqty = inbound.qty-item['sum_qty']
            print(" id:",inbound.id," seller:",inbound.seller," Product:",inbound.item," Code:",inbound.code," Active:",inbound.active," Qty:",updateqty)
        
        


In [ ]:
create_inbound_document(mock_inbound=[{'seller':1,'product':2,'code': 'A0001','quantity':1000}])

In [ ]:
check_inbound_quality(mock_product= [{
        'product':2, 
        'qty':500},
    {
        'product':2, 
        'qty':200},
    {
        'product':2, 
        'qty':300}
    ])

In [ ]:
checkout_shipping_cart( cart = [{
        'item': 1, 
        'qty': 5},
    ])

In [ ]:
create_picking_list(sale_orders=[21])

In [ ]:
create_packing_list(sale_orders=[21])

In [ ]:
Move.objects.all().delete()
Inbound.objects.all().delete()
PickItem.objects.all().delete()
PackItem.objects.all().delete()
PickRequest.objects.all().delete()
PackRequest.objects.all().delete()
OrderItem.objects.all().delete()
SaleOrder.objects.all().delete()

In [ ]:
PickItem.objects.all().delete()
PackItem.objects.all().delete()
PickRequest.objects.all().delete()
PackRequest.objects.all().delete()
OrderItem.objects.all().delete()
SaleOrder.objects.all().delete()

In [ ]:
# list_product_stok
location = Location.objects.get(pk=3)
product = Move.objects.filter(src_location=location.pk).values(
    'Product__id',
    'Product__name',
    'Product__product_sub_category__subname',
    'Product__description',
    'Product__price_per_unit',
    'qty',
    'Product__unit',
)
pd.DataFrame(product)

In [ ]:
list_stock()